In [1]:
ENV["GKS_ENCODING"]="utf-8"
using Statistics
using Dates, TimeZones
using CSV, DataFrames, DataFramesMeta
using JuliaDB
using Plots, Plots.PlotMeasures
using PyPlot
using Formatting
using StatsPlots
using PyCall

pyplot()

Plots.PyPlotBackend()

In [2]:
function get_font(size::Integer)
    Plots.font(family="Noto Sans KR", pointsize=size)
end

font12 = get_font(12)
font14 = get_font(14)
font18 = get_font(18)
font24 = get_font(24)
font32 = get_font(32)
default(titlefont=font32, guidefont=font24, tickfont=font24, legendfont=font24)

In [3]:
stn_codes = [111121, 111123, 111131, 111141, 111142,
    111151, 111152, 111161, 111171, 111181,
    111191, 111201, 111212, 111221, 111231,
    111241, 111251, 111261, 111262, 111273,
    111274, 111281, 111291, 111301, 111311]
stn_names = ["중구", "종로구", "용산구", "광진구", "성동구",
    "중랑구", "동대문구", "성북구", "도봉구", "은평구",
    "서대문구", "마포구", "강서구", "구로구", "영등포구",
    "동작구", "관악구", "강남구", "서초구", "송파구",
    "강동구", "금천구", "강북구", "양천구", "노원구"]
stations = JuliaDB.table((code = stn_codes, name = stn_names))
jongro_stn_code = 111123
jongro_stn_name = "종로구"

"종로구"

In [4]:
case_name = "1910_OptimumS5"
output_size = 24
zoom_mindate = DateTime(2018, 1, 1, 1)
zoom_maxdate = DateTime(2018, 12, 31, 23)
zoom_sdate = DateTime(2018, 7, 1, 1)
zoom_fdate = DateTime(2018, 7, 10, 23)
zoom_sdate2 = DateTime(2018, 11, 1, 1)
zoom_fdate2 = DateTime(2018, 11, 10, 23)

2018-11-10T23:00:00

In [5]:
feature_csv = "/home/appleparan/data/" * case_name * "/post/feature/feature_stats.csv"
station_csv = "/home/appleparan/data/" * case_name * "/post/station/station_stats.csv"
forecast_csv = "/home/appleparan/data/" * case_name * "/post/forecast/forecast_stats.csv"
pearson_corr_csv = "/home/appleparan/data/" * case_name * "/pearson_corr.csv"

"/home/appleparan/data/1910_OptimumS5/pearson_corr.csv"

In [6]:
ENV["GKSwstype"] = "100"
ENV["GKS_ENCODING"] = "utf-8"
const BG_COLOR = RGB(255/255, 255/255, 255/255)
const LN_COLOR = RGB(67/255, 75/255, 86/255)
const MK_COLOR = RGB(67/255, 75/255, 86/255)
const LN01_COLOR = RGB(202/255,0/255,32/255)
const LN02_COLOR = RGB(5/255,113/255,176/255)
const FL01_COLOR = RGB(239/255, 138/255, 98/255)
const FL02_COLOR = RGB(103/255, 169/255, 207/255)
const datefmt = "yyyymmddHH"    

"yyyymmddHH"

# Zoomed Plot

In [7]:
function plot_DNN_lineplot(stn_code, stn_name, s_date::DateTime, f_date::DateTime, ycol::Symbol)
    output_size = 24
    for hour in 1:24 
        hour_pad = lpad(hour, 2, '0')
        output_dir = "/home/appleparan/data/" * case_name * "/post/station/$(hour_pad)/"
        csv_path = output_dir *
            string(ycol) * "_" *
            string(stn_code) * "_" *
            stn_name * "_plottable_$(hour_pad)h.csv"
        plot_path = output_dir *
            string(ycol) * "_" *
            string(stn_code) * "_" *
            stn_name * "_" * 
            Dates.format(s_date + Dates.Hour(1), datefmt) * "_"  *
            Dates.format(f_date + Dates.Hour(1), datefmt) * "_$(hour_pad)h_line"
        
        # load data
        df_raw = CSV.read(csv_path)
        df = @linq df_raw |>
            where(:date .>= s_date + Dates.Hour(hour) , :date .<= f_date + Dates.Hour(hour))

        gr(size = (2560, 1080))
        pl1 = Plots.plot(df[:, :date], df[:, :y],
            ylim = (0.0, 
                max(maximum(df[:, :y]), maximum(df[:, :yhat]))),
            line=:solid, linewidth=5, label="obs.",
            guidefontsize = 24, titlefontsize = 32, tickfontsize = 24, legendfontsize = 24, margin=15px,
            guidefontcolor = LN_COLOR, titlefontcolor = LN_COLOR, tickfontcolor = LN_COLOR, legendfontcolor = LN_COLOR,
            background_color = BG_COLOR, color=LN01_COLOR,
            title="$(String(ycol)) in dates ($(hour)h) at " * stn_name, 
            xlabel="date", ylabel=String(ycol), legend=:top)
        pl1 = Plots.plot!(df[:, :date], df[:, :yhat],
            line=:solid, linewidth=5, color=LN02_COLOR, label="model")
        Plots.png(pl1, plot_path * ".png")
        Plots.svg(pl1, plot_path * ".svg")
    end
end

plot_DNN_lineplot (generic function with 1 method)

In [ ]:
for ycol in [:PM25, :PM10]
    for stn in stations
        @show ycol, stn[:name]
        # quartely plot
        plot_DNN_lineplot( stn[:code], stn[:name], DateTime(2018, 1, 4, 1), DateTime(2018, 3, 31, 23), ycol)
        plot_DNN_lineplot( stn[:code], stn[:name], DateTime(2018, 4, 1, 1), DateTime(2018, 6, 30, 23), ycol)
        plot_DNN_lineplot( stn[:code], stn[:name], DateTime(2018, 7, 1, 1), DateTime(2018, 9, 30, 23), ycol)
        plot_DNN_lineplot( stn[:code], stn[:name], DateTime(2018, 10, 1, 1), DateTime(2018, 12, 31, 23), ycol)
        
        # 10 days plot
        plot_DNN_lineplot( stn[:code], stn[:name], zoom_sdate, zoom_fdate, ycol)
        plot_DNN_lineplot( stn[:code], stn[:name], zoom_sdate2, zoom_fdate2, ycol)
    end
end

(ycol, stn[:name]) = (:PM25, "중구")
(ycol, stn[:name]) = (:PM25, "종로구")
(ycol, stn[:name]) = (:PM25, "용산구")
(ycol, stn[:name]) = (:PM25, "광진구")
(ycol, stn[:name]) = (:PM25, "성동구")
(ycol, stn[:name]) = (:PM25, "중랑구")
(ycol, stn[:name]) = (:PM25, "동대문구")
(ycol, stn[:name]) = (:PM25, "성북구")
(ycol, stn[:name]) = (:PM25, "도봉구")
(ycol, stn[:name]) = (:PM25, "은평구")


# Station Line Plot

In [ ]:
#gr(size = (1920, 1080))

In [ ]:
function plot_eval_per_station(evalprop::Symbol, base_stn_code::Integer, base_stn_name::String, ycol::Symbol)
    df = CSV.read(station_csv)
    output_dir = "/home/appleparan/data/" * case_name * "/post/station/"
    plot_path = output_dir * "$(String(evalprop))_$(String(ycol))_station"

    df_ycol = @linq df |> where(:colname .== String(ycol), :code .!= base_stn_code)
    df_ycol_base = @linq df |> where(:colname .== String(ycol), :code .== base_stn_code)
    df_ycol_eval = df_ycol[:, evalprop]
    df_ycol_eval_base = df_ycol_base[:, evalprop]
    
    val_ycol_eval_base = df_ycol_base[1, evalprop]
    names_evals = df_ycol[:, :name]
    codes_evals = df_ycol[:, :code]
    arr_evals = df_ycol[:, evalprop]

    pl = Plots.bar(names_evals, arr_evals, yformatter = :plain,
        xrotation=45, color=[FL01_COLOR FL02_COLOR],
        xticks = (0.5:1:(length(names_evals)-0.5), names_evals),
        xtickfontsize = 18, ytickfontsize = 24, margin=50px,
        guidefontsize = 24, titlefontsize = 32, legendfontsize = 24,
        guidefontcolor = LN_COLOR, titlefontcolor = LN_COLOR, tickfontcolor = LN_COLOR, legendfontcolor = LN_COLOR,
        background_color = BG_COLOR, 
        title=string(evalprop),
        xlabel="station", ylabel=string(evalprop), legend=false)
    
    Plots.hline!([val_ycol_eval_base], line=(4,FL01_COLOR), label="$(String(ycol))($(base_stn_name))")
    Plots.png(pl, plot_path * ".png")
    Plots.svg(pl, plot_path * ".svg")
    #Plots.plot(pl)
end

In [ ]:
plot_eval_per_station(:RMSE, jongro_stn_code, jongro_stn_name, :PM10)
plot_eval_per_station(:RMSE, jongro_stn_code, jongro_stn_name, :PM25)

In [ ]:
function plot_corr_per_station(base_stn_code::Integer, base_stn_name::String, ycol::Symbol)
    output_size = 24
    output_dir = "/home/appleparan/data/" * case_name * "/post/station/"

    hours = collect(1:24)
    for ycol in [:PM25, :PM10]
        df = DataFrame(hour = hours)
        pl_arr = []

        for stn in stations
            csv_path = output_dir * "$(String(ycol))_$(stn[:code])_$(stn[:name])_corr.csv"
            df_stn = CSV.read(csv_path)
            df_stn = DataFrames.rename(df_stn, :corr => Symbol(:corr, "_$(stn[:name])"))
            df = DataFrames.join(df, df_stn, on = :hour)
            
        end

        df_stn_corr = DataFrame(hours = Int64[], name = String[], code = Int64[], corr = Float64[])

        for stn in stations
            name = stn[:name]
            code = stn[:code]
            column_name = Symbol("corr_$(String(name))")
            df_stn_sub = df[:, [:hour, column_name]]
            df_stn_tmp = DataFrame(hours = df_stn_sub[:, :hour],
                    name = repeat([String(name)], output_size),
                    code = repeat([code], output_size),
                    corr = df_stn_sub[:, column_name])
            
            df_stn_corr = vcat(df_stn_corr, df_stn_tmp)
        end
        
        for hour = 1:output_size
            df_corr = @linq df_stn_corr |> where(:hours .== hour, :code .!= base_stn_code)
            df_corr_base = @linq df_stn_corr |> where(:hours .== hour, :code .== base_stn_code)
            df_corr_eval = df_corr[:, :corr]
            df_corr_eval_base = df_corr_base[:, :corr]
            
            arr_evals = df_corr_eval
            val_corr_base = df_corr_eval_base
            names_evals = df_corr[:, :name]
            codes_evals = df_corr[:, :code]

            plot_path = output_dir * "corr_$(String(ycol))_station_$(hour)h"

            pl = Plots.bar(names_evals, arr_evals, yformatter = :plain,
                xrotation=45, color=[FL01_COLOR FL02_COLOR],
                xticks = (0.5:1:(length(names_evals)-0.5), names_evals),
                xtickfontsize = 18, ytickfontsize = 24, margin=50px,
                guidefontsize = 24, titlefontsize = 32, legendfontsize = 24,
                guidefontcolor = LN_COLOR, titlefontcolor = LN_COLOR, tickfontcolor = LN_COLOR, legendfontcolor = LN_COLOR,
                background_color = BG_COLOR, 
                title="Correlation in $(hour)h",
                xlabel="Station", ylabel="Correlation", legend=false)
            Plots.hline!(val_corr_base, line=(4,FL01_COLOR), label="$(String(ycol))($(base_stn_name))")
            Plots.png(pl, plot_path * ".png")
            Plots.svg(pl, plot_path * ".svg")
            #Plots.plot(pl)
            push!(pl_arr, pl)
        end
        #=
        anim = @animate for i=1:length(pl_arr)
            Plots.plot(pl_arr[i])
        end
        gif(anim,output_dir * "corr_$(String(ycol))_station.gif", fps=2)
        =#
    end
end

In [ ]:
plot_corr_per_station(jongro_stn_code, jongro_stn_name, :PM10)
plot_corr_per_station(jongro_stn_code, jongro_stn_name, :PM25)

In [14]:
function plot_eval_per_feature(case_name_feas_prefix::String, evalprop::Symbol, ycol::Symbol)
    df = CSV.read(feature_csv)
    output_dir = "/home/appleparan/data/" * case_name_feas_prefix * "/post/feature/"
    plot_path = output_dir * "$(String(evalprop))_$(String(ycol))_feature"

    df_ycol = @linq df |> where(:colname .== String(ycol), :rm_fea .!= "FULL")
    df_ycol_full = @linq df |> where(:colname .== String(ycol), :rm_fea .== "FULL")

    val_ycol_evals = df_ycol[:, evalprop]
    val_ycol_evals_full = df_ycol_full[1, evalprop]
    
    rm_features = df_ycol[:, :rm_fea]
    evals_arr = val_ycol_evals

    pl = Plots.bar(rm_features, evals_arr,
        yformatter = :plain,
        guidefontsize = 24, titlefontsize = 32, tickfontsize = 24, legendfontsize = 24, margin=15px,
        guidefontcolor = LN_COLOR, titlefontcolor = LN_COLOR, tickfontcolor = LN_COLOR, legendfontcolor = LN_COLOR,
        background_color = BG_COLOR, color=[FL01_COLOR FL02_COLOR],
        title=string(evalprop), 
        xlabel="station", ylabel=string(evalprop), legend=:false)
    hline!([val_ycol_evals_full], line=(4,FL01_COLOR), label="$(String(ycol)) (Full)")

    Plots.png(pl, plot_path * ".png")
    Plots.svg(pl, plot_path * ".svg")
    #Plots.plot(pl)
end

plot_eval_per_feature (generic function with 1 method)

In [96]:
suffices = [:no_CO2,:no_O3,:no_NO2, :no_SO2, :no_temp, :no_uandv, :no_prepandsnow, :no_humid]
dir_prefix = ["no" * String(s) for s in suffices]
#for p in dir_prefix
p = "1907_rewrite"
plot_eval_per_feature(p, :IOA, :PM10)
plot_eval_per_feature(p, :IOA, :PM25)
#end

In [97]:
function plot_corr_per_feature(case_name_feas_prefix::String, ycol::Symbol, features)
    
    output_dir = "/home/appleparan/data/" * case_name_feas_prefix * "/post/feature/"

    hours = collect(1:24)
    
    for ycol in [:PM25, :PM10]
        df = DataFrame(hour = hours)
        plot_path = output_dir * "corr_$(String(ycol))_rm_"

        for fea in features
            csv_path = output_dir * "$(String(ycol))_rm_$(String(fea))_corr.csv"
            df_fea = CSV.read(csv_path)
            df_fea = DataFrames.rename(df_fea, :corr => Symbol(:corr, "_rm_$(fea)"))
            df = DataFrames.join(df, df_fea, on = :hour)
        end

        df_fea_corr = DataFrame(hours = Int64[], feature = String[], corr = Float64[])

        for fea in features
            column_name = Symbol("corr_rm_$(String(fea))")
            df_fea_sub = df[:, [:hour, column_name]]
            df_fea_tmp = DataFrame(hours = df_fea_sub[:, :hour],
                    feature = repeat([String(fea)], output_size),
                    corr = df_fea_sub[:, column_name])
            
            df_fea_corr = vcat(df_fea_corr, df_fea_tmp)
        end

        for hour = 1:output_size
            df_corr = @linq df_fea_corr |> where(:hours .== hour, :feature .!= "FULL")
            df_corr_base = @linq df_fea_corr |> where(:hours .== hour, :feature .== "FULL")
            df_corr_eval = df_corr[:, :corr]
            df_corr_eval_base = df_corr_base[:, :corr]
            
            arr_evals = df_corr_eval
            val_corr_base = df_corr_eval_base
            fea_evals = df_corr[:, :feature]

            plot_path = output_dir * "corr_$(String(ycol))_feature_$(hour)h"
            gr(size = (2560, 1080))
            pl = Plots.bar(fea_evals, arr_evals, yformatter = :plain,
                xrotation=45, color=[FL01_COLOR FL02_COLOR],
                guidefontsize = 24, titlefontsize = 32, tickfontsize = 24, legendfontsize = 24, margin=15px,
                guidefontcolor = LN_COLOR, titlefontcolor = LN_COLOR, tickfontcolor = LN_COLOR, legendfontcolor = LN_COLOR,
                background_color = BG_COLOR, 
                title="Correlation in $(hour)h",
                xlabel="Features", ylabel="Correlation", legend=false)
            Plots.hline!(val_corr_base, line=(4,FL01_COLOR), label="$(String(ycol))_FULL")
            Plots.png(pl, plot_path * ".png")
            Plots.svg(pl, plot_path * ".svg")
            Plots.plot(pl)
        end
    end
end

plot_corr_per_feature (generic function with 3 methods)

In [99]:
features = [:SO2, :CO, :O3, :NO2, :PM10, :PM25, :temp, :uv, :pres, :prepsnow, :humid, :FULL]
plot_corr_per_feature("1907_rewrite", :PM10, features)
plot_corr_per_feature("1907_rewrite", :PM25, features)

In [88]:
function show_forecast()
    df = CSV.read(forecast_csv)
   
    df_PM10 = @linq df |> where(:colname .== "PM10")
    df_PM25 = @linq df |> where(:colname .== "PM25")

    @show "PM10 (all & high) : ", Statistics.mean(df_PM10[:, :fore_all]), Statistics.mean(df_PM10[:, :fore_high]) 
    @show "PM25 (all & high) : ", Statistics.mean(df_PM25[:, :fore_all]), Statistics.mean(df_PM25[:, :fore_high]) 
end

show_forecast (generic function with 1 method)

In [89]:
show_forecast()

("PM10 (all & high) : ", Statistics.mean(df_PM10[:, :fore_all]), Statistics.mean(df_PM10[:, :fore_high])) = ("PM10 (all & high) : ", 0.7821560480147737, 0.8028131562073)
("PM25 (all & high) : ", Statistics.mean(df_PM25[:, :fore_all]), Statistics.mean(df_PM25[:, :fore_high])) = ("PM25 (all & high) : ", 0.808213296398892, 0.815966765300334)


("PM25 (all & high) : ", 0.808213296398892, 0.815966765300334)

In [12]:
using Statistics
function plot_corr()
    df = CSV.read(pearson_corr_csv)
    output_dir = "/home/appleparan/data/" * case_name * "/post/"
    plot_path = output_dir * "pearson_corr"
    
    dfM = convert(Matrix, df[:, :])
    dfm_cor = Statistics.cor(dfM)
    feas = names(df)
    
    ann = []
    for i in 1:length(feas)
    for j in 1:length(feas)
        _ann = (i - 0.5, j - 0.5, Plots.text(format( df[i, j], precision=2), 18, :white))
        push!(ann, _ann)
    end
    end
    
    crpl = Plots.heatmap(string.(feas), string.(feas), dfm_cor,
        annotations = ann,
        clim = (-1.0, 1.0), c=:blues, legend=true,
        guidefontsize = 18, titlefontsize = 24, tickfontsize = 18, legendfontsize = 18, margin=15px,
        guidefontcolor = LN_COLOR, titlefontcolor = LN_COLOR, tickfontcolor = LN_COLOR, legendfontcolor = LN_COLOR,
        title="CORR", background_color = BG_COLOR)

    Plots.plot(crpl)
    Plots.png(crpl, plot_path * ".png")
    Plots.svg(crpl, plot_path * ".svg")
end


plot_corr (generic function with 1 method)

In [13]:
plot_corr()

In [100]:
1+1

2